In [2]:
# built in modules
import pandas as pd
import numpy as np
from sklearn import metrics 

# .py files 
from preprocess import Preprocess
from feature_extraction import TFIDF
from feature_extraction import SGLstm

# Can use this library to reload a specific module if the notebook can't see changes in the imported module
import importlib
from torch import nn
from keras.models import Sequential
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Dropout


c:\Users\emada\AppData\Local\Programs\Python\Python310\lib\site-packages\farasa\__base.py:157: UserWarning: You are using old version of java. Farasa is compatiple with Java 7 and above 
  warnings.warn(
[2022-12-26 14:18:07,643 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
[2022-12-26 14:18:10,468 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
c:\Users\emada\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
t = pd.read_csv('./Dataset/train.csv')
d = pd.read_csv('./Dataset/dev.csv')

In [4]:
preprocess = Preprocess()

Emojis:  False
Lemmatizor:  farasapy


In [5]:
categories = ["info_news", "celebrity", "plan", "requests", "rumors", "advice", "restrictions", "personal", "unrelated", "others"]

stance = ["0", "1", "2"] # 0: -ve, 1: 0, 2: +ve

In [6]:
# we have 9 categories so we have an array of len 9 for each y
# def encode_category(y):
#     '''
#     Input: y a list of string labels for the category of each document 
#     Output: a list of encoded 10 sized array for the category of each doc 
#             for "other" category , it has an array =[0 0 0 0 0 0 0 0 0 1] 
#     '''
#     return [categories.index(ele) for ele in y]

In [7]:
# # we have 9 categories so we have an array of len 9 for each y
# def encode_stance(y):
#     '''
#     Input: y a list of string labels for the category of each document 
#     Output: a list of encoded 10 sized array for the category of each doc 
#             for "other" category , it has an array =[0 0 0 0 0 0 0 0 0 1] 
#     ''' 
#     return [stance.index(ele) for ele in y]

In [8]:
X_beforeTokenization = t['text']
ys = t['stance']
yc = t['category']

X_beforeTokenization_dev = d['text']
ys_dev = d['stance']
yc_dev = d['category']

In [9]:
# yc=encode_category(yc)

# do I need this?
# ys=encode_stance(ys)

# yc_dev=encode_category(yc_dev)
# ys_dev=encode_stance(ys_dev)

# Preprocess the test data
X = X_beforeTokenization.apply(preprocess.do_all)
X_dev = X_beforeTokenization_dev.apply(preprocess.do_all)

In [10]:
# all words in the dataset 
lst = [word for x in X for word in x]
print(len(lst))

# unique words
vocab = set(lst)
print(len(vocab))

649553
16526


In [11]:
# Load the model
# sg_model = KeyedVectors.load_word2vec_format('./Word2VecSkipGram300D.bin', binary=True)

In [12]:
max_length=max([len(s) for s in X])
print(max_length)

604


In [18]:
# SG features
train_features_sg,test_features_sg=SGLstm(train_documents=X,test_documents=X_dev,max_len=max_length)


In [19]:
print(len(train_features_sg))
print(np.shape(train_features_sg[1]))

6988
(126, 300)


In [15]:
# itf-idf features
# train_features_tfidf,test_features_tfidf=TFIDF(train_documents=X,test_documents=X_dev)
# print(train_features_tfidf.shape)

In [ ]:
# to apply sequential model, need to pass inputs of the same size to the model 
# so we need to pad the input to the same size
# we will pad the input to the maximum length of the input

# get the maximum length of the input
max_length=max([len(s) for s in train_features_sg])
print(max_length)
print(len(train_features_sg[0][0]))

train_features_sg_arr= np.zeros(len(train_features_sg))
# All sentences should be padded to have the same length
for i in range(len(train_features_sg)):
    train_features_sg_arr[i]=train_features_sg[i]+['<PAD>'] * (max_length - len(X[i]))
    # train_features_sg[i][0].extend(['<PAD>'] * (max_length - len(X[i])))
# print([ys[0]])
# ys_list=[]
# ys_list.append(ys)
# ys_list[0].extend([0] * 1)
# ys_list[1].extend([0] * 1)
# print(ys_list)
# for i in range(len(ys)):
#     ys[i].extend([0] * (max_length - len(ys[i])))
# for i in range(len(yc)):
#     yc[i].extend([0] * (max_length - len(yc[i])))

In [17]:
# word vector
print(len(train_features_sg[0][0]))
print(train_features_sg_arr[0])
print(len(train_features_sg_arr[0][0]))

300


In [ ]:
print(len(train_features_sg))
print(train_features_sg[1])

In [19]:
# Reshape the data to be 3D
# print(train_features_sg.shape)
# # number of tweets, length of each tweet, number of features 
# train_features_sg_3d= np.reshape(train_features_sg, (len(train_features_sg), train_features_sg.shape[1], 1))
# print(train_features_sg_3d.shape)

In [ ]:

# def initLSTM():
vocab_size = len(vocab)
embedding_dim = 300 #need to be tuned
hidden_size = 50 #need to be tuned

embedding = nn.Embedding(vocab_size, embedding_dim)

lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)

n_classes_category = 9
n_classes_stance = 3

linear_category = nn.Linear(hidden_size, n_classes_category)
linear_stance = nn.Linear(hidden_size, n_classes_stance)


In [ ]:
# lstm = torch.nn.LSTM(input_size=300, hidden_size=50, num_layers=1, batch_first=True)

In [24]:
# Build the model for stances
lstm_model = Sequential()
lstm_model.add(LSTM(128, input_shape=(max_length, 300), return_sequences=False))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(64))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(lstm_model.summary())
ys = np.array(ys) + 1

# Fit the lstm_model
history = lstm_model.fit(train_features_sg, ys, epochs=12, batch_size=32, verbose=1)

ValueError: Input 0 of layer lstm_5 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 128)

In [ ]:
y_pred = lstm_model.predict(test_features_sg)

In [ ]:
# Calculate the F1 score for each class
from sklearn.metrics import f1_score
print("F1 score for each class -> ",f1_score(ys, y_pred.argmax(axis=1), average=None))
# Calculate the Macro Average F1 score for the whole data
print("Macro Average F1 score -> ",f1_score(ys, y_pred.argmax(axis=1), average='macro'))